In [1]:
import sys
print(sys.version)

3.7.6 (default, Jan  8 2020, 19:59:22) 
[GCC 7.3.0]


In [2]:
# run all code cells of the specified notebook
%run /notebooks/local/libs/crawler-class.ipynb

In [3]:
BASE_URL = "https://www.thelastofus.playstation.com/"

crawler = Crawler(BASE_URL)
soup = crawler.parseUrl(BASE_URL, False)

print(soup.title)

<title>The Last of Us Part II Game | PS4 - PlayStation</title>
